# HILL CLIMBING ALGORITHIM A CART POLE

In [3]:
import gym
import random
import numpy as np
import os
from gym.envs.registration import register
import time
from IPython.display import clear_output
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import adam

env_name = "CartPole-v1"
env = gym.make(env_name)
print("Observation space:", env.observation_space)
print("Action space:", env.action_space)
#for continuous action space





Using TensorFlow backend.


Observation space: Box(4,)
Action space: Discrete(2)


In [4]:
class QNAgent():
    def __init__(self,env,discount_rate=0.97,learning_rate=0.01,randomness=1,qtable1=0):
        self.action_size = env.action_space.n
        self.state_size = env.observation_space.shape[0]
        print("Action size: {} , State size: {}".format(self.action_size,self.state_size))
        
        #our model was not exploring other actions, so fix
        #probability of exploring priortizing exploratory action over the policy action
        self.eps=1.0
        
        self.discount_rate=discount_rate
        self.learning_rate=learning_rate
        self.randomness=randomness

   
        self.build_model()
        self.replay_buffer = deque(maxlen=10000000)
        self.random_count=0
        self.policy_count=0
            
    #defining the member function
    #defining rows as state and coloumns as actions
    def build_model(self):
        
        self.model =Sequential()
        self.model.add(Dense(units=24,activation='relu',input_dim=4))
        self.model.add(Dense(units=24,activation='relu'))
        self.model.add(Dense(units=self.action_size,activation='linear'))
        self.model.compile(loss='mse',optimizer=adam(lr=self.learning_rate))
                  

    #overwriting the get action function
    def get_action(self,state,ep,i):
        
        q_state = self.model.predict(state)
        #to perform the explortory actions, first we are storing the obvious action
        action_greedy= np.argmax(q_state[0])
        #here we are storing the random action, which is defined in the parent class
        action_random=random.choice(range(self.action_size))
                       
        #below retuens which position is max
        
        #1. calculating a random value
        #2. if it is less than 1 (that we will decay exponentilly)
        
        if self.randomness==1:
            kk=random.random()
            if kk<self.eps:
                #print('random')
                self.random_count+=1
                return action_random
            else:
                #print('Policy')
                self.policy_count+=1
                return action_greedy

        else:
            return action_greedy
    
    def train(self,experience,reached,batch_size= 50):
        #if experience[4]:
        #    self.eps=self.eps*0.99
        #self.replay_buffer.append(experience)
        #if len(self.replay_buffer) < batch_size: 
            #print('The buffer lenght is {}'.format(len(self.replay_buffer)))
         #   return
        #samples = random.sample(self.replay_buffer,batch_size) 
        # samples.append(experience)
        samples=experience
        if reached:
            gamma=1
        else:
            gamma=0.99
        reward_cumm=0
        for state, action, state_next, reward, done in reversed(samples):
    
           
            reward_cumm = reward_cumm + reward*gamma
            q_target = reward_cumm
            q_next =  self.model.predict(state_next)
            q_value = self.model.predict(state)
            if not done:
                q_target= reward_cumm + self.discount_rate*np.max(q_next[0])
  
            q_value[0][action] = q_target
            #print("q_target before: {} , q_target after: {}".format(q_target,q_value[0][action]))
            if not reached:
                gamma=gamma*0.99
            
            
            self.model.fit(state, q_value, verbose=0)
        self.eps=self.eps*0.99

                #print('before {} after {}'.format(kk,self.eps))


        
agent= QNAgent(env)

Action size: 2 , State size: 4






In [5]:
total_reward=0
for ep in range(500):
    state = env.reset()
    done=False
    total_reward=0
    i=0
    episode_training=[]
    while not done:
    #   action = env.action_space.sample()
        state = np.reshape(state,[1,4])
        action = agent.get_action(state,ep,i)
        next_state, reward, done, info = env.step(action)
        next_state = np.reshape(next_state,[1,4])
        
        
        episode_training.append((state,action,next_state,reward,done))
        #print(done)
        state=next_state
        total_reward+=reward
        #for displaying the popup window
        env.render()
        i+=1
        #clear_output(wait=True)
    print("Episode: {}, Total reward: {}, eps: {}, Random Count: {},Policy Count: {}".format(ep,total_reward,agent.eps,agent.random_count,agent.policy_count))
    if i>=200:
        agent.train(episode_training,True)
    else:
        agent.train(episode_training,False)
    agent.random_count=0
    agent.policy_count=0
        
agent.model.save("model.h5")   
        

env.close()



Episode: 0, Total reward: 25.0, eps: 1.0, Random Count: 25,Policy Count: 0
Episode: 1, Total reward: 11.0, eps: 0.99, Random Count: 11,Policy Count: 0
Episode: 2, Total reward: 12.0, eps: 0.9801, Random Count: 12,Policy Count: 0
Episode: 3, Total reward: 19.0, eps: 0.9702989999999999, Random Count: 19,Policy Count: 0
Episode: 4, Total reward: 27.0, eps: 0.96059601, Random Count: 25,Policy Count: 2
Episode: 5, Total reward: 35.0, eps: 0.9509900498999999, Random Count: 32,Policy Count: 3
Episode: 6, Total reward: 15.0, eps: 0.9414801494009999, Random Count: 13,Policy Count: 2
Episode: 7, Total reward: 40.0, eps: 0.9320653479069899, Random Count: 38,Policy Count: 2
Episode: 8, Total reward: 19.0, eps: 0.92274469442792, Random Count: 19,Policy Count: 0
Episode: 9, Total reward: 17.0, eps: 0.9135172474836407, Random Count: 14,Policy Count: 3
Episode: 10, Total reward: 23.0, eps: 0.9043820750088043, Random Count: 20,Policy Count: 3
Episode: 11, Total reward: 10.0, eps: 0.8953382542587163, 

KeyboardInterrupt: 